# Matrix completion

In [1]:
pip install chompack

     |████████████████████████████████| 8.8 MB 8.2 MB/s eta 0:00:01     |██████▎                         | 1.7 MB 8.2 MB/s eta 0:00:01     |██████████████████████████████  | 8.2 MB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 12.7 MB 16.5 MB/s eta 0:00:01     |████▊                           | 1.9 MB 16.5 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [289]:
from cvxopt import spmatrix, matrix, spdiag, sparse, amd
import chompack as cp
import random

## Rules
- $n < N$
- $N' = N + n$ 
- $u < N < u+n$
- $u+n < N'$

## Set properties

In [281]:
random.seed(4242)

N = 10

n_full = [x for x in range(2, N)]
n = random.choice(n_full)

u = []
for i in range(N):
    u.append(i) if n + i > N else None

N_prime = N + n

In [282]:
print("N : ", N)
print("n : ", n)
print("u : ", u)
print("N' : ", N_prime)

N :  10
n :  8
u :  [3, 4, 5, 6, 7, 8, 9]
N' :  18


### Autom

In [332]:
# generate sparse matrix
N_prime = 7
N = 5
n = 2
u = 4

#N_matrix = matrix(
#    [[x for x in range(N)] for x in range(N)]
#)
N_matrix = matrix(
    [x for x in range(N*N)], (N,N)
)
#n_matrix = matrix(
#    [[N+x for x in range(n)] for x in range(n)]
#)
n_matrix = matrix(
    [x for x in range(n*n)], (n,n)
)
#u_matrix = matrix(
#    [[N+x for x in range(u+n)] for x in range(u+n)]
#)
u_matrix = matrix(
    [x for x in range(u*u)], (u,u)
)

P_matrix = spdiag([N_matrix, n_matrix])

nu_line = [x for x in range(N-u, N_prime)]
nu_col = [x for x in range(N-u, N_prime)]
P_matrix[nu_line, nu_col] = 8

print("N : \n", N_matrix)
print("n : \n", n_matrix)
print("u : \n", u_matrix)

print(P_matrix)

N : 
 [  0   5  10  15  20]
[  1   6  11  16  21]
[  2   7  12  17  22]
[  3   8  13  18  23]
[  4   9  14  19  24]

n : 
 [ 0  2]
[ 1  3]

u : 
 [  0   4   8  12]
[  1   5   9  13]
[  2   6  10  14]
[  3   7  11  15]

[ 0.00e+00  5.00e+00  1.00e+01  1.50e+01  2.00e+01     0         0    ]
[ 1.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00]
[ 2.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00]
[ 3.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00]
[ 4.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00]
[    0      8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00]
[    0      8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00  8.00e+00]



In [333]:
# compute symbolic factorization
symb = cp.symbolic(P_matrix, p=amd.order)
print(symb)
print(symb.sparsity_pattern(reordered=False, symmetric=False))

[X X X X     X]
[X X X X X X X]
[X X X X X X X]
[X X X X X X X]
[  X X X X X X]
[  X X X X X X]
[X X X X X X X]

[ 1.00e+00     0         0         0         0         0         0    ]
[ 1.00e+00  1.00e+00     0         0         0         0         0    ]
[ 1.00e+00  1.00e+00  1.00e+00     0         0         0         0    ]
[ 1.00e+00  1.00e+00  1.00e+00  1.00e+00     0         0         0    ]
[ 1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00     0         0    ]
[    0      1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00     0    ]
[    0      1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00]



In [334]:
L = cp.cspmatrix(symb)
L += P_matrix

In [335]:
cp.cholesky(L)
print(L)

ArithmeticError: factorization failed

## Test

In [336]:
N_prime = 7
N = 5
n = 2
u = 4

T_matrix = matrix(
    [x+1 for x in range(N_prime*N_prime)], (N_prime,N_prime)
)

T_matrix[[0], [5, 6]] = 0
T_matrix[[5, 6], [0]] = 0
print(T_matrix)

[  1   8  15  22  29   0   0]
[  2   9  16  23  30  37  44]
[  3  10  17  24  31  38  45]
[  4  11  18  25  32  39  46]
[  5  12  19  26  33  40  47]
[  0  13  20  27  34  41  48]
[  0  14  21  28  35  42  49]



In [337]:
# compute symbolic factorization
test_matrix = sparse([T_matrix])
symb_test = cp.symbolic(test_matrix, p=amd.order)
print(test_matrix)
print(symb_test)
print(symb.sparsity_pattern(reordered=False, symmetric=False))

L_test = cp.cspmatrix(symb_test)
L_test += test_matrix

cp.cholesky(L_test)
print(L_test)

[ 1.00e+00  8.00e+00  1.50e+01  2.20e+01  2.90e+01     0         0    ]
[ 2.00e+00  9.00e+00  1.60e+01  2.30e+01  3.00e+01  3.70e+01  4.40e+01]
[ 3.00e+00  1.00e+01  1.70e+01  2.40e+01  3.10e+01  3.80e+01  4.50e+01]
[ 4.00e+00  1.10e+01  1.80e+01  2.50e+01  3.20e+01  3.90e+01  4.60e+01]
[ 5.00e+00  1.20e+01  1.90e+01  2.60e+01  3.30e+01  4.00e+01  4.70e+01]
[    0      1.30e+01  2.00e+01  2.70e+01  3.40e+01  4.10e+01  4.80e+01]
[    0      1.40e+01  2.10e+01  2.80e+01  3.50e+01  4.20e+01  4.90e+01]

[X X X X     X]
[X X X X X X X]
[X X X X X X X]
[X X X X X X X]
[  X X X X X X]
[  X X X X X X]
[X X X X X X X]

[ 1.00e+00     0         0         0         0         0         0    ]
[ 1.00e+00  1.00e+00     0         0         0         0         0    ]
[ 1.00e+00  1.00e+00  1.00e+00     0         0         0         0    ]
[ 1.00e+00  1.00e+00  1.00e+00  1.00e+00     0         0         0    ]
[ 1.00e+00  1.00e+00  1.00e+00  1.00e+00  1.00e+00     0         0    ]
[    0      1.00e+00  

ArithmeticError: factorization failed